In [1]:
import numpy as np
import glob
import os
import json
import _pickle as cPickle
import nltk

In [4]:
path = os.path.join('/srv/scratch1/fnoora', "l2l_semantic_index.pkl")
semantic_index = cPickle.load(open(path, "rb"))
path = os.path.join('/srv/scratch1/fnoora', "l2l_semantic.pkl")
semantic_dict = cPickle.load(open(path, "rb"))

In [5]:
def evaluate_wordnet(lang, preds_list, truth_dict, semantic_dict, pred2label):
    score = 0.
    count = 0
    scores={'normal':[],
            'syn':[],
            'syn+hyp':[],
            'syn+hyp+hpo':[]}
    label_2_predict={}
    syn_score=0
    syn_hyp_score=0
    syn_hyp_hpo_score=0
    
    for entry in preds_list:
        quesid = entry["questionId"]
        pred = entry["prediction"]
        try:
            label = truth_dict[quesid]["answer"]
            if pred == label:
                score += 1.
            elif pred in semantic_dict[label]['hpo'] :
                syn_hyp_hpo_score += 1.
#                 if lang=='en': 
                #print('label:',label,'hypo->',pred,':pred')
                if f'{label}:{pred} (hpo)' in pred2label:
                    pred2label[f'{label}:{pred} (hpo)']+=1
                else:
                    pred2label[f'{label}:{pred} (hpo)']=1
                    
            elif pred in semantic_dict[label]['hyp']  :
                syn_hyp_score += 1.
#                 if lang=='en': 
                    #print('label:',label,'hypr->',pred,':pred')
                if f'{label}:{pred} (hyp)' in pred2label:
                    pred2label[f'{label}:{pred} (hyp)']+=1
                else:
                    pred2label[f'{label}:{pred} (hyp)']=1
            elif pred in semantic_dict[label]['syn'] :
                syn_score += 1.
                # collect the confused labels for each lang
                    #print('label:',label,'syn->',pred,':pred')
                    #pred2label[f'{label}:{pred}']='syn'
                if f'{label}:{pred} (syn)' in pred2label:
                    pred2label[f'{label}:{pred} (syn)']+=1
                else:
                    pred2label[f'{label}:{pred} (syn)']=1
                
            count += 1
        except:
            pass
    
    syns=syn_score
#     hyps=syn_hyp_score
#     hpos=syn_hyp_hpo_score
    syn_score+=score
#     syn_hyp_score+=syn_score
#     syn_hyp_hpo_score+=syn_hyp_score
    
    scores={'normal':(score / count)*100,
            'syn':[syns, (syn_score/count)*100],
#             '+hypr':[hyps, (syn_hyp_score/count)*100],
#             '+hypo':[hpos,(syn_hyp_hpo_score/count)*100]}
           }
    
    return scores

In [6]:
import json
lang_scores={}
TASK=15
MODEL='uc2'
MODEL_CONFIG='uc2_base'
TASKS_CONFIG='iglue_test_tasks_boxes36.dtu'
TRTASK='GQA'
USE='zero_shot_sft_en_task_semanticlosss_top10_global_seed256_wo_cls_zero_masking_mix_in_sft'
pred2label_in_lang_best={}
for LANG in ['en','bn','de','id','ko','pt','ru','zh']:#'id','ko','pt','ru','zh'
    TETASK=f'xGQA{LANG}'
    pred2label={}
    TEXT_PATH=f"/home/user/fnoora/iglue/datasets/xGQA/annotations/few_shot/{LANG}/test.json"
    OUTPUT_DIR=f'/srv/scratch1/fnoora/iglue/{USE}/xgqa/{MODEL}/{TRTASK}_{MODEL_CONFIG}/{TETASK}/test'
    preds_file=f'{OUTPUT_DIR}/pytorch_model_best.bin-/test_{LANG}_result.json'
    preds_list = json.load(open(preds_file))
    truth_dict = json.load(open(TEXT_PATH))
    scores=evaluate_wordnet(LANG,preds_list,truth_dict,semantic_dict,pred2label)
    lang_scores[LANG]=scores
    pred2label_in_lang_best[LANG]=pred2label

lang_scores

{'en': {'normal': 55.30836092715232, 'syn': [179.0, 57.16059602649006]},
 'bn': {'normal': 32.191639072847686, 'syn': [114.0, 33.37127483443709]},
 'de': {'normal': 50.444950331125824, 'syn': [160.0, 52.100579470198674]},
 'id': {'normal': 41.84602649006622, 'syn': [165.0, 43.553394039735096]},
 'ko': {'normal': 39.828228476821195, 'syn': [186.0, 41.75289735099338]},
 'pt': {'normal': 43.812086092715234, 'syn': [129.0, 45.14693708609271]},
 'ru': {'normal': 44.27773178807947, 'syn': [157.0, 45.902317880794705]},
 'zh': {'normal': 46.14031456953642, 'syn': [235.0, 48.57201986754967]}}

In [7]:
[v['normal'] for k,v in lang_scores.items()]

[55.30836092715232,
 32.191639072847686,
 50.444950331125824,
 41.84602649006622,
 39.828228476821195,
 43.812086092715234,
 44.27773178807947,
 46.14031456953642]

In [8]:
lang_scores={}
pred2label_in_lang={}
model='uc2'
for LANG in ['en','bn','de','id','ko','pt','ru','zh']:#'id','ko','pt','ru','zh'
    TETASK=f'xGQA{LANG}'
    pred2label={}
    TEXT_PATH=f"/home/user/fnoora/iglue/datasets/xGQA/annotations/few_shot/{LANG}/test.json"
    #OUTPUT_DIR=f'/srv/scratch1/fnoora/iglue/{USE}/xgqa/{MODEL}/{TRTASK}_{MODEL_CONFIG}/{TETASK}/test'
    preds_file=f'/home/user/fnoora/iglue/results/xgqa/zero_shot/{model}/test_{LANG}_0_result.json'
    preds_list = json.load(open(preds_file))
    truth_dict = json.load(open(TEXT_PATH))
    scores=evaluate_wordnet(LANG,preds_list,truth_dict,semantic_dict,pred2label)
    lang_scores[LANG]=scores
    pred2label_in_lang[LANG]=pred2label

lang_scores

{'en': {'normal': 55.20488410596026, 'syn': [164.0, 56.90190397350994]},
 'bn': {'normal': 19.991721854304636, 'syn': [9.0, 20.084850993377483]},
 'de': {'normal': 42.860099337748345, 'syn': [119.0, 44.09147350993378]},
 'id': {'normal': 28.67342715231788, 'syn': [77.0, 29.47019867549669]},
 'ko': {'normal': 21.367963576158942, 'syn': [3.0, 21.399006622516556]},
 'pt': {'normal': 30.42218543046358, 'syn': [63.0, 31.07408940397351]},
 'ru': {'normal': 31.001655629139073, 'syn': [91.0, 31.94329470198676]},
 'zh': {'normal': 31.16721854304636, 'syn': [84.0, 32.0364238410596]}}

In [9]:
import pandas as pd
confsued_in_lang={}
confsued_in_lang_n={}
all_issues=[]
use=pred2label_in_lang_best
for lang in use:
    k_most_confused=sorted( use[lang].items(), key=lambda pair: -pair[1] )[:5]
    #df = pd.DataFrame(k_most_confused, columns =['issue','number'])
    issues=[x[0] for x in list(k_most_confused)]
    numbers=[x[1] for x in list(k_most_confused)]
    issues += ['-'] * (5 - len(issues))
    numbers +=  ['-'] * (5 - len(numbers))
#     print(*issues)
#     print(*numbers)
    confsued_in_lang[lang]=issues
    confsued_in_lang_n[lang]=numbers
    all_issues.extend(issues)
    

df1_best = pd.DataFrame.from_dict(confsued_in_lang, orient='index')
df1_best_n = pd.DataFrame.from_dict(confsued_in_lang_n, orient='index')


In [10]:
df1_best

,0,1,2,3,4
en,girl:woman (hyp),material:color (hpo),cabinets:cabinet (syn),woman:girl (hpo),zebras:zebra (syn)
bn,cabinets:cabinet (syn),girl:woman (hyp),skater:skateboarder (hpo),woman:girl (hpo),lady:woman (hyp)
de,girl:woman (hyp),material:color (hpo),lady:woman (hyp),cabinets:cabinet (syn),woman:girl (hpo)
id,girl:woman (hyp),cabinets:cabinet (syn),woman:girl (hpo),chairs:chair (syn),lady:woman (hyp)
ko,cabinets:cabinet (syn),girl:woman (hyp),elephants:elephant (syn),woman:girl (hpo),chairs:chair (syn)
pt,material:color (hpo),girl:woman (hyp),woman:girl (hpo),zebras:zebra (syn),lady:woman (hyp)
ru,girl:woman (hyp),cabinets:cabinet (syn),material:color (hpo),woman:girl (hpo),lady:woman (hyp)
zh,cabinets:cabinet (syn),girl:woman (hyp),chairs:chair (syn),zebras:zebra (syn),elephants:elephant (syn)


In [149]:
df1_best_n

,0,1,2,3,4
en,28,24,20,18,16
bn,29,19,15,12,12
de,32,23,18,17,16
id,27,24,17,17,17
ko,39,34,20,17,17
pt,25,24,20,15,15
ru,33,25,19,18,16
zh,32,27,26,25,24


In [11]:
import pandas as pd
confsued_in_lang={}
confsued_in_lang_n={}
all_issues=[]
use=pred2label_in_lang
for lang in use:
    k_most_confused=sorted( use[lang].items(), key=lambda pair: -pair[1] )[:5]
    #df = pd.DataFrame(k_most_confused, columns =['issue','number'])
    issues=[x[0] for x in list(k_most_confused)]
    numbers=[x[1] for x in list(k_most_confused)]
    issues += ['-'] * (5 - len(issues))
    numbers +=  ['-'] * (5 - len(numbers))
#     print(*issues)
#     print(*numbers)
    confsued_in_lang[lang]=issues
    confsued_in_lang_n[lang]=numbers
    all_issues.extend(issues)
    

df2_best = pd.DataFrame.from_dict(confsued_in_lang, orient='index')
df2_best_n = pd.DataFrame.from_dict(confsued_in_lang_n, orient='index')

In [12]:
df2_best

,0,1,2,3,4
en,girl:woman (hyp),material:color (hpo),lady:woman (hyp),coffee table:table (hyp),zebras:zebra (syn)
bn,sailboats:sailboat (syn),skater:skateboarder (hpo),plain:field (syn),trees:tree (syn),tank top:shirt (hyp)
de,girl:woman (hyp),material:color (hpo),lady:woman (hyp),woman:girl (hpo),street sign:sign (hyp)
id,girl:woman (hyp),lady:woman (hyp),skater:skateboarder (hpo),woman:girl (hpo),zebras:zebra (syn)
ko,girl:woman (hyp),skater:skateboarder (hpo),boy:man (hyp),fire truck:truck (hyp),gown:dress (hyp)
pt,girl:woman (hyp),skater:skateboarder (hpo),lady:woman (hyp),zebras:zebra (syn),woman:girl (hpo)
ru,girl:woman (hyp),skater:skateboarder (hpo),lady:woman (hyp),woman:girl (hpo),cabinets:cabinet (syn)
zh,girl:woman (hyp),chairs:chair (syn),cabinets:cabinet (syn),skater:skateboarder (hpo),lady:woman (hyp)


In [13]:
df2_best_n

,0,1,2,3,4
en,27,23,18,17,16
bn,3,3,2,2,1
de,33,21,16,13,13
id,28,18,15,14,12
ko,7,7,2,2,1
pt,22,17,13,12,11
ru,32,17,17,14,12
zh,26,15,15,15,15


In [14]:
import json
lang_scores={}
TASK=15
MODEL='m3p'
MODEL_CONFIG='m3p_base'
TASKS_CONFIG='iglue_test_tasks_boxes36.dtu'
TRTASK='GQA'
USE='m3p_zero_shot_seed_semanticloss_sft41'
pred2label_in_lang_best={}
for LANG in ['en','bn','de','id','ko','pt','ru','zh']:#'id','ko','pt','ru','zh'
    TETASK=f'xGQA{LANG}'
    pred2label={}
    TEXT_PATH=f"/home/user/fnoora/iglue/datasets/xGQA/annotations/few_shot/{LANG}/test.json"
    OUTPUT_DIR=f'/srv/scratch1/fnoora/iglue/{USE}/xgqa/{MODEL}/{TRTASK}_{MODEL_CONFIG}/{TETASK}/test'
    preds_file=f'{OUTPUT_DIR}/pytorch_model_best.bin-/test_{LANG}_result.json'
    preds_list = json.load(open(preds_file))
    truth_dict = json.load(open(TEXT_PATH))
    scores=evaluate_wordnet(LANG,preds_list,truth_dict,semantic_dict,pred2label)
    lang_scores[LANG]=scores
    pred2label_in_lang_best[LANG]=pred2label


In [174]:
lang_scores

{'en': {'normal': 56.14652317880795, 'syn': [154.0, 57.74006622516556]},
 'bn': {'normal': 22.557947019867548, 'syn': [77.0, 23.354718543046356]},
 'de': {'normal': 40.58360927152318, 'syn': [100.0, 41.61837748344371]},
 'id': {'normal': 27.27649006622517, 'syn': [147.0, 28.79759933774834]},
 'ko': {'normal': 34.79925496688742, 'syn': [155.0, 36.40314569536424]},
 'pt': {'normal': 38.741721854304636, 'syn': [105.0, 39.828228476821195]},
 'ru': {'normal': 39.48675496688742, 'syn': [133.0, 40.862996688741724]},
 'zh': {'normal': 35.834023178807946, 'syn': [150.0, 37.386175496688736]}}